# Recommender system

In [3]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import seaborn as sns 

## 1.1  Movielens full - EDA

- a) Gör en EDA för att förstå datasetet. Inkludera olika slags plots. Begränsa dig inte till frågorna nedan, utan försök undersöka fler aspekter av datan. 

In [4]:
#load data and get some info about it
movies = pd.read_csv("Data/movies.csv")
ratings= pd.read_csv("C:/Users/Amal Derbali/Desktop/ratings.csv")

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [9]:
#try to list all genres existed in the list

Genre_list=[]
for genres in movies["genres"].unique(): #get unique genre from the list and avoid repetition
        Genre_list.extend(genres.split("|"))
Genre_list= set(Genre_list) #set the list
Genre_list

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [6]:
#try count the number of films in every genre
num_per_genre=movies['genres'].groupby(movies['genres']).count()
num_per_genre

genres
(no genres listed)                     4266
Action                                  531
Action|Adventure                        195
Action|Adventure|Animation               37
Action|Adventure|Animation|Children      12
                                       ... 
Thriller|War                             13
Thriller|Western                          5
War                                      82
War|Western                               6
Western                                 551
Name: genres, Length: 1643, dtype: int64

- b) Vilka är de 10 filmerna med flest ratings?

In [7]:
ratings_by_title=ratings.merge(movies, on='movieId') #use merge to put titles and ratings at the same data
ratings_by_title

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,6,307,4.0,832059248,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,56,307,4.0,1383625728,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,71,307,5.0,1257795414,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
4,84,307,3.0,999055519,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
...,...,...,...,...,...,...
27753439,282403,167894,1.0,1524243885,Stranglehold (1994),Action
27753440,282732,161572,3.5,1504408070,The Great Houdini (1976),Drama
27753441,283000,117857,3.5,1417317969,Hotline (2014),Documentary
27753442,283000,133409,3.5,1431539331,Barnum! (1986),(no genres listed)


In [8]:
Top_ratings= ratings_by_title.groupby('title').size().sort_values(ascending=False)[:10]
Top_ratings

title
Shawshank Redemption, The (1994)             97999
Forrest Gump (1994)                          97040
Pulp Fiction (1994)                          92406
Silence of the Lambs, The (1991)             87899
Matrix, The (1999)                           84545
Star Wars: Episode IV - A New Hope (1977)    81815
Jurassic Park (1993)                         76451
Schindler's List (1993)                      71516
Braveheart (1995)                            68803
Toy Story (1995)                             68469
dtype: int64

- c) Beräkna den genomsnittliga ratingen för dessa 10 filmerna med flest ratings.

In [ ]:
# get mean of ratings for top 10 movies
mean=ratings_by_title.groupby(ratings_by_title['title'].mean())
mean=mean.loc[Top_ratings.index] #get titles of the top rated movies

In [9]:
# add year column to the data
movies["year"]= movies["title"].str.extract(r"(\d+)")#extracting data based on : https://community.dataquest.io/t/str-extract-r-d-what-does-d-mean/506096
movies["year"]= movies["title"].str.extract(r".*\((.*)\)")
movies

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed),1946
58094,193878,Les tribulations d'une caissière (2011),Comedy,2011
58095,193880,Her Name Was Mumu (2016),Drama,2016
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi,2017


In [ ]:
#create a plot of movies per year
movies_yr=movies.groupby("year").count()
movies_yr